In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import xgboost as xgb
import pandas as pd
import numpy as np

### IMPORTANTE: correr el archivo Data Processing.ipynb que se encuentra en la carpeta Data/ antes de leer los .csv

In [2]:
train = pd.read_csv("Data/train_processed_tf_idf_bow.csv")
test = pd.read_csv("Data/test_processed_tf_idf_bow.csv")
sub_sample = pd.read_csv("Data/sample_submission.csv")

print (train.shape, test.shape, sub_sample.shape)

(7613, 1844) (3263, 1843) (3263, 2)


## XGBoost

Como parte del preprocesamiento se escalan los datos aplicando normalizacion y como modelo XGBoost

In [ ]:
features_to_drop = ['id', 'keyword','location','text','location_clean','text_clean', 'hashtags', 'mentions','links']

X_train = train.drop(columns = features_to_drop + ['target'])
X_test = test.drop(columns = features_to_drop)
y_train = train.target

clf = xgb.XGBClassifier(max_depth=200, n_estimators=400, subsample=1, learning_rate=0.07, reg_lambda=0.1, reg_alpha=0.1,\
                       gamma=1)
clf.fit(X_train, y_train)

y_test = clf.predict(X_train)

submit = sub_sample.copy()
submit.target = y_test
submit.to_csv('submit_xgboost.csv',index=False)

In [ ]:
# F-1 score
print ("Training set f1_score :", np.round(f1_score(y_train, predictions),5))

In [ ]:
# Matriz de Confucion
pd.DataFrame(confusion_matrix(y_train, clf.predict(X_train)))

## Evaluacion del Modelo

In [ ]:
# Cross validation
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=123)
cv_score = cross_val_score(clf, X_train, y_train, cv=cv, scoring='f1')
print('Cross validation F-1 score: %.3f' %np.mean(cv_score))